In [ ]:
!nvidia-smi #### check it's T4

Wed Jul 14 13:51:17 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import os
!git clone https://github.com/ibrahim-elsawy/dstilPegasus
# !pip install -r /content/konafa/colab_req.txt
!pip install datasets
#!pip install torch==1.7.1
!pip install transformers==4.3.0rc1
!pip install sentencepiece 
!pip install catalyst==20.6
!pip install rouge-score
!pip install sacrebleu

Cloning into 'dstilPegasus'...
remote: Enumerating objects: 427, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 427 (delta 13), reused 2 (delta 2), pack-reused 400
Receiving objects: 100% (427/427), 156.12 KiB | 4.73 MiB/s, done.
Resolving deltas: 100% (271/271), done.
     |████████████████████████████████| 266kB 8.7MB/s 
     |████████████████████████████████| 245kB 46.4MB/s 
     |████████████████████████████████| 122kB 49.2MB/s 
     |████████████████████████████████| 2.8MB 7.4MB/s 
     |████████████████████████████████| 901kB 44.4MB/s 
     |████████████████████████████████| 3.2MB 36.0MB/s 
     |████████████████████████████████| 1.2MB 7.6MB/s 
     |████████████████████████████████| 368kB 8.1MB/s 
     |████████████████████████████████| 133kB 14.4MB/s 
     |████████████████████████████████| 51kB 6.6MB/s 
     |████████████████████████████████| 174kB 16.4MB/s 
     |████████████████████████████████| 71kB 9.1MB/s

In [ ]:
os.chdir('/content/dstilPegasus')
import torch
import transformers
from catalyst import dl
from src.runners import DistilMLMRunner
from src.runners import DistilMLMRunnerFT
# from src.models import DistilpegasusStudentModel, PegasusForMLM #BertForMLM  DistilbertStudentModel
from catalyst.core import MetricAggregationCallback
from torch.utils.data import DataLoader
from src.callbacks import (
    CosineLossCallback,
    KLDivLossCallback,
    MaskedLanguageModelCallback,
    MSELossCallback,
    PerplexityMetricCallbackDistillation,
    SmoothingLossCallback,
    CrossentropylossCallback
)
from utils.rouge import calc_generative_metrics 
import pandas as pd
from typing import Iterable, Union

# import torch
from torch.utils.data import Dataset
from tqdm.auto import tqdm
# import transformers
from transformers import AutoTokenizer
from transformers.data.data_collator import DataCollatorForLanguageModeling
import numpy as np
from transformers.data.data_collator import default_data_collator
from torch import nn
from typing import Dict, List
from datasets import load_dataset
import datasets
from datasets import Dataset

/usr/local/lib/python3.7/dist-packages/tqdm/std.py:658: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
/usr/local/lib/python3.7/dist-packages/jsonschema/compat.py:6: DeprecationWarning:

Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working

/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject

/usr/local/lib/python3.7/dist-packages/yaml/constructor.py:126: DeprecationWarning:

Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working

/usr/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning:

numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 

In [ ]:
import os 
from google.colab import drive
drive.mount("/content/drive")
os.chdir("/content/drive/MyDrive")

Mounted at /content/drive


In [ ]:
# train_dataset = datasets.load_from_disk("/content/drive/MyDrive/pl/data/xsumtrainDataset_4") ###train_dataset = datasets.load_from_disk("/content/drive/MyDrive/pl/data/xsumtrainDataset_2")
# valid_dataset = datasets.load_from_disk("/content/drive/MyDrive/pl/data/xsumvalidDataset")
# # train_dataset = datasets.load_from_disk("/content/drive/MyDrive/orig/xsumtrainDataset") ###train_dataset = datasets.load_from_disk("/content/drive/MyDrive/pl/data/xsumtrainDataset_2")
# # valid_dataset = datasets.load_from_disk("/content/drive/MyDrive/orig/xsumvalidDataset")
test_dataset = datasets.load_from_disk("/content/drive/MyDrive/orig/xsumtestDataset")
teacher = torch.load('teacher_16_4.pt', map_location=torch.device('cuda')) #torch.load('teacher_model.pt', map_location=torch.device('cuda'))

In [ ]:
!nvidia-smi

Wed Jul 14 13:53:10 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P0    30W /  70W |   2484MiB / 15109MiB |      2%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
import datasets
import warnings
import torch
from torch import nn
from typing import Optional, Tuple, List, Union
from transformers import PegasusModel, PegasusConfig, PegasusForConditionalGeneration
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, PreTrainedModel
from transformers import SummarizationPipeline
students_config_book = {
    '2': PegasusConfig(encoder_layers=2, decoder_layers=2),
    '4': PegasusConfig(encoder_layers=4, decoder_layers=4),
    '6': PegasusConfig(encoder_layers=6, decoder_layers=6),
    '8': PegasusConfig(encoder_layers=8, decoder_layers=8),
    '10': PegasusConfig(encoder_layers=10, decoder_layers=10),
    '12': PegasusConfig(encoder_layers=12, decoder_layers=12),
    '16': PegasusConfig(encoder_layers=16, decoder_layers=16)
}


LAYERS_TO_COPY = {
    16: {  # maps  num layers in student -> which teacher layers to copy
        1: [0],
        2: [0, 15],
        3: [0, 8, 15],
        4: [0, 5, 10, 15],
        6: [0, 3, 6, 9, 12, 15],
        8: [0, 2, 4, 6, 8, 10, 12, 15],
        9: [0, 1, 3, 5, 7, 9, 11, 13, 15],
        12: [0, 1, 2, 3, 4, 5, 6, 7, 9, 11, 13, 15],
        16: list(range(16)),
    },
}
LAYERS_TO_SUPERVISE = {
    # maps  num layers in student -> which teacher layers to copy.
    16: {1: [15], 4: [4, 9, 12, 15], 8: [1, 3, 5, 7, 9, 11, 13, 15]},
}


def copy_layers(src_layers: nn.ModuleList, dest_layers: nn.ModuleList, layers_to_copy) -> None:
    layers_to_copy = nn.ModuleList([src_layers[i] for i in layers_to_copy])
    assert len(dest_layers) == len(
        layers_to_copy), f"{len(dest_layers)} != {len(layers_to_copy)}"
    dest_layers.load_state_dict(layers_to_copy.state_dict())

# Copied from transformers.models.bart.modeling_bart.shift_tokens_right


def shift_tokens_right(input_ids: torch.Tensor, pad_token_id: int, decoder_start_token_id: int):
    """
    Shift input ids one token to the right.
    """
    shifted_input_ids = input_ids.new_zeros(input_ids.shape)
    shifted_input_ids[:, 1:] = input_ids[:, :-1].clone()
    shifted_input_ids[:, 0] = decoder_start_token_id

    assert pad_token_id is not None, "self.model.config.pad_token_id has to be defined."
    # replace possible -100 values in labels by `pad_token_id`
    shifted_input_ids.masked_fill_(shifted_input_ids == -100, pad_token_id)

    return shifted_input_ids


def pick_layers_to_copy(n_student, n_teacher):
    try:
        val = LAYERS_TO_COPY[n_teacher][n_student]
        return val
    except KeyError:
        if n_student != n_teacher:
            warnings.warn(
                f"no hardcoded layers to copy for teacher {n_teacher} -> student {n_student}, defaulting to first {n_student}"
            )
        return list(range(n_student))


def get_layers_to_supervise(n_student, n_teacher) -> List[int]:
    """Used or the --supervise_forward kwarg"""
    if n_student > n_teacher:
        raise ValueError(
            f"Cannot perform intermediate supervision for student {n_student} > teacher {n_teacher}")
    elif n_teacher == n_student:
        return list(range(n_teacher))
    elif n_student == 1:
        return [n_teacher - 1]
    else:
        return LAYERS_TO_SUPERVISE[n_teacher][n_student]


def create_student_with_configuration(teacher,
                                      e=None,
                                      d=None,
                                      copy_first_teacher_layers = False,
                                      save_path='./student'):

    teacher.eval()
    teacher_e, teacher_d = teacher.config.encoder_layers, teacher.config.decoder_layers
    init_kwargs = teacher.config.to_diff_dict()
    if e is None:
        e = teacher_e
    if d is None:
        d = teacher_d
    init_kwargs.update({"encoder_layers": e, "decoder_layers": d})
    student_cfg = teacher.config_class(**init_kwargs)
    student = AutoModelForSeq2SeqLM.from_config(student_cfg)
    # Start by copying the full teacher state dict this will copy the first N teacher layers to the student.
    info = student.load_state_dict(teacher.state_dict(), strict=False)
    # every student key should have a teacher keys.
    assert info.missing_keys == [], info.missing_keys

    if copy_first_teacher_layers:  # Our copying is done. We just log and save
        e_layers_to_copy, d_layers_to_copy = list(range(e)), list(range(d))
        student.save_pretrained(save_path)
        return student, e_layers_to_copy, d_layers_to_copy

    # Decide which layers of the teacher to copy. Not exactly alternating -- we try to keep first and last layer.
    e_layers_to_copy: List[int] = pick_layers_to_copy(e, teacher_e)
    d_layers_to_copy: List[int] = pick_layers_to_copy(d, teacher_d)

    copy_layers(teacher.model.encoder.layers,
                student.model.encoder.layers, e_layers_to_copy)
    copy_layers(teacher.model.decoder.layers,
                student.model.decoder.layers, d_layers_to_copy)

    student.config.init_metadata = dict(
        teacher_type=teacher.config.model_type,
        copied_encoder_layers=e_layers_to_copy,
        copied_decoder_layers=d_layers_to_copy,
    )
    #student.save_pretrained(save_path)
    # Save information about copying for easier reproducibility

    return student


In [ ]:
student = create_student_with_configuration(teacher,
                                     e=6,
                                     d=4,
                                     copy_first_teacher_layers = False,
                                     save_path='./student').to('cuda')
# student = teacher
model = torch.nn.ModuleDict({"student": student})
checkpoint = torch.load('trained_student_6_4_noTA.pt', map_location=torch.device('cuda'))
student.load_state_dict(checkpoint['model_state_dict'])
del teacher

In [ ]:
torch.cuda.empty_cache()
!nvidia-smi

Wed Jul 14 06:45:08 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P0    29W /  70W |   3642MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
del checkpoint
torch.cuda.empty_cache()
!nvidia-smi

Wed Jul 14 06:45:09 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   60C    P0    32W /  70W |   2710MiB / 15109MiB |     17%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
#train_dataset.set_format('torch', columns=['input_ids', 'attention_mask'], device='cuda')
#valid_dataset.set_format('torch', columns=['input_ids', 'attention_mask'], device='cuda')
test_data = Dataset.from_dict(test_dataset[0:])

test_data.set_format('torch', columns=['input_ids', 'attention_mask','decode_ids','decode_mask'], device='cuda')

In [ ]:
# train_dataloader = DataLoader(
#     #train_dataset['train']
#     train_data, batch_size=8
# )
test_dataloader = DataLoader(
    test_data, batch_size=1
)
# loaders = {"train": train_dataloader, "valid": valid_dataloader}
# del train_dataset
# del valid_dataset
#teacher = torch.load('teacher_model.pt', map_location=torch.device('cuda')).to('cuda')
#student = torch.load('st_3dec_3enc.pt', map_location=torch.device('cuda')).to('cuda')
#model = torch.nn.ModuleDict({"teacher": teacher, "student": student})

In [ ]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer
import pickle
import time
import numpy as np
model_name = "google/pegasus-xsum"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
# dict_scores = {"rouge1":[], "rouge2":[], "rougeL":[], "rougeLsum":[],"all_time" : []}
pred = []
labels = []
inf_time = []
#a_file = open("rouge_scores_1.pkl", "rb")
#dict_scores = pickle.load(a_file)
#a_file.close()
student.eval()
for i,batch in enumerate(test_dataloader):
  batch['decode_ids'] = batch['decode_ids'].to('cuda')
  batch['input_ids'] = batch['input_ids'].to('cuda')
  batch['attention_mask'] = batch['attention_mask'].to('cuda')
  labels.append(tokenizer.batch_decode(batch['decode_ids'],skip_special_tokens=True)[0])
  time_before = time.time()
  pred_ids = student.generate(input_ids = batch['input_ids'],attention_mask = batch['attention_mask'],max_length=60,use_cache=True )
  time_after = time.time()
  pred.append(tokenizer.batch_decode(pred_ids,skip_special_tokens=True)[0])
  inf_time.append((time_after-time_before)/batch["input_ids"].shape[0])
  if i%10 == 0:
    print(i)
scores = calc_generative_metrics(pred,labels)
print(scores)
print(np.average(inf_time[:]))

/usr/local/lib/python3.7/dist-packages/torch/_tensor.py:575: UserWarning:

floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  /pytorch/aten/src/ATen/native/BinaryOps.cpp:467.)



0
10
20
30
40
50
60
70
80
90
100
110
120
130
140
150
160
170
180
190
200
210
220
230
240
250
260
270
280
290
300
310
320
330
340
350
360
370
380
390
400
410
420
430
440
450
460
470
480
490
500
510
520
530
540
550
560
570
580
590
600
610
620
630
640
650
660
670
680
690
700
710
720
730
740
750
760
770
780
790
800
810
820
830
840
850
860
870
880
890
900
910
920
930
940
950
960
970
980
990
1000
1010
1020
1030
1040
1050
1060
1070
1080
1090
1100
1110
1120
1130
1140
1150
1160
1170
1180
1190
1200
1210
1220
1230
1240
1250
1260
1270
1280
1290
1300
1310
1320
1330
1340
1350
1360
1370
1380
1390
1400
1410
1420
1430
1440
1450
1460
1470
1480
1490
1500
1510
1520
1530
1540
1550
1560
1570
1580
1590
1600
1610
1620
1630
1640
1650
1660
1670
1680
1690
1700
1710
1720
1730
1740
1750
1760
1770
1780
1790
1800
1810
1820
1830
1840
1850
1860
1870
1880
1890
1900
1910
1920
1930
1940
1950
1960
1970
1980
1990
2000
2010
2020
2030
2040
2050
2060
2070
2080
2090
2100
2110
2120
2130
2140
2150
2160
2170
2180
2190
2200
2210
2

In [ ]:
def get_response(input_text):
  batch = tokenizer([input_text],truncation=True,padding='longest',max_length=1000, return_tensors="pt").to('cuda')
  translated = student.generate(**batch,max_length=60,use_cache=True ,num_return_sequences=1)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text

In [ ]:
context1 = "The problem is affecting people using the older versions of the PlayStation 3, called the 'Fat' model.The problem isn't affecting the newer PS3 Slim systems that have been on sale since September last year.Sony have also said they are aiming to have the problem fixed shortly but is advising some users to avoid using their console for the time being.'We hope to resolve this problem within the next 24 hours,' a statement reads. 'In the meantime, if you have a model other than the new slim PS3, we advise that you do not use your PS3 system, as doing so may result in errors in some functionality, such as recording obtained trophies, and not being able to restore certain data.'We believe we have identified that this problem is being caused by a bug in the clock functionality incorporated in the system.'The PlayStation Network is used by millions of people around the world.It allows users to play their friends at games like Fifa over the internet and also do things like download software or visit online stores."
print(get_response(context1))
context2 = "She will play Denker, a lady’s maid to Dame Maggie Smith’s character, the Dowager Countess of Grantham. Johnston, who has also appeared in Waking the Dead and Coronation Street, joins new stars Richard E Grant and Anna Chancellor, both of whom will play guests of the Granthams at Downton. The hit period drama will return to screens this autumn. Series four of the show, which followed the wealthy Grantham family and their servants, achieved an average of 11.9 million viewers in the UK. The very British drama has also been a huge hit in the US, winning both Emmy Awards and Golden Globes. More than 26 million viewers watched series four on Masterpiece on PBS, making it one of the highest rating shows on American television. Previous high profile guest stars include Shirley Maclaine who played Martha Levinson, Lady Grantham’s mother, and Oscar-nominated actor Paul Giamatti who appeared in last year’s Christmas special as her ”maverick, playboy” son. Series five will also feature 24 star Rade Sherbedgia as a Russian refugee who has fled the revolution after World War 1. Earlier this year, executive producer Gareth Neame promised it would have ”all the usual highs and lows, romance, drama and comedy”."
print(get_response(context2))
context3 = "Media playback is not supported on this device Craig Cathcart put the visitors ahead before substitute Simon Church won and scored an 89th-minute penalty. ”There were lots of positives out of it even if we’d have come off and lost 1-0. They had a good mentality and attitude,” said Coleman. Wales face another Euro 2016 warm-up game against Ukraine in Kiev on Monday. ”We look forward to our next challenge now,” added Coleman. ”The team will change up again, and we’ll see how they go again.” Striker Church, currently on loan at Scottish Premiership side Aberdeen from Reading, was delighted with his equaliser from the spot. ”Northern Ireland were a tough side to play against. They’ve obviously done well to get where they are and it was a tough game,” he said. ”We wanted to do well because it was the last time a Wales crowd would see us before the Euros and we wanted to put in a good performance. ”I’ve just got to keep going now and hopefully score some goals. This is a great squad to be part of.”"
print(get_response(context3))

['Sony has warned users that they are unable to use their PlayStation 3 console because of a bug in the clock.']
['Actress Joanna Johnston has joined the cast of Downton Abbey in the fifth series of ITV’s Downton Abbey.']
['Wales manager Chris Coleman praised his side’s attitude after they drew 1-1 with Northern Ireland in their final warm-up game before Euro 2016.']
